# 📈 03 – Advanced Visualizations & Rolling Analysis

## Overview

This notebook extends the factor modeling analysis with **time-varying** perspectives and **advanced visualizations**:

1. **Rolling Window Analysis**: Examine how model fit (R²) changes over time
2. **Dynamic Factor Sensitivity**: Track evolving relationships between EM and macro factors  
3. **Interactive Visualizations**: Generate comprehensive charts for all EM indices
4. **Reusable Functions**: Modular code for reproducible analysis

### Key Features:
- **Rolling R² Analysis**: 60-day rolling window regression models
- **Time-Series Visualization**: Dynamic model performance tracking
- **Batch Processing**: Automated chart generation for all EM indices
- **Export Functionality**: Save all visualizations to output folder

### Use Cases:
- **Risk Management**: Identify periods of high/low factor sensitivity
- **Portfolio Analysis**: Understand when diversification benefits change
- **Market Timing**: Spot regime changes in EM-macro relationships

## 📦 Import Required Libraries

Loading libraries for advanced analysis and visualization:

In [ ]:
# Core data manipulation
import pandas as pd
import numpy as np
import os

# Machine learning components  
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style for professional appearance
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

## 📁 Data Loading & Preparation

Load the dataset and prepare variables for rolling analysis.

In [ ]:
# Load the combined dataset
df = pd.read_csv('../data/combined_em_macro_data.csv', parse_dates=['date'], index_col='date')

# Convert to log returns
log_returns = np.log(df / df.shift(1)).dropna()

# Separate EM and macro variables
em_cols = [c for c in df.columns if c.startswith(('Brazil', 'India', 'China', 'SouthAfrica', 'Mexico', 'Indonesia'))]
macro_cols = [c for c in df.columns if c not in em_cols]

Y_all = log_returns[em_cols]    # EM equity returns
X_all = log_returns[macro_cols] # Macro factor returns

print(f"📊 Dataset loaded for rolling analysis:")
print(f"   • Time period: {df.index.min()} to {df.index.max()}")
print(f"   • Total observations: {len(log_returns)}")
print(f"   • EM indices: {len(em_cols)}")
print(f"   • Macro factors: {len(macro_cols)}")

print(f"\n🌏 EM Indices: {em_cols}")
print(f"📈 Macro Factors: {macro_cols}")

## 🔄 Rolling Window Analysis Function

Create a reusable function to perform rolling window PCA-regression analysis.

In [ ]:
def rolling_r2_scores(X, Y, window=60, n_components=3):
    """
    Calculate rolling R² scores for EM indices using PCA-based factor models.
    
    Parameters:
    -----------
    X : pd.DataFrame
        Macro factor returns (independent variables)
    Y : pd.DataFrame  
        EM equity returns (dependent variables)
    window : int
        Rolling window size in days (default: 60)
    n_components : int
        Number of principal components to use (default: 3)
    
    Returns:
    --------
    pd.DataFrame
        Rolling R² scores for each EM index
    """
    
    # Initialize results DataFrame
    results = pd.DataFrame(index=Y.index[window:], columns=Y.columns)
    
    print(f"🔄 Computing rolling R² with {window}-day windows...")
    print(f"   • Total windows: {len(Y) - window + 1}")
    print(f"   • PCA components: {n_components}")
    
    # Loop through each EM index
    for col_idx, col in enumerate(Y.columns):
        print(f"   • Processing {col} ({col_idx + 1}/{len(Y.columns)})")
        
        # Loop through time windows
        for i in range(window, len(Y)):
            # Extract window data
            X_window = X.iloc[i - window:i]
            Y_window = Y[col].iloc[i - window:i]
            
            try:
                # Standardize macro factors
                scaler = StandardScaler()
                X_scaled = scaler.fit_transform(X_window)
                
                # Apply PCA
                pca = PCA(n_components=n_components)
                X_pca = pca.fit_transform(X_scaled)
                
                # Fit regression model
                model = LinearRegression().fit(X_pca, Y_window)
                
                # Store R² score
                results.at[Y_window.index[-1], col] = model.score(X_pca, Y_window)
                
            except Exception as e:
                # Handle potential numerical issues
                results.at[Y_window.index[-1], col] = np.nan
    
    print("✅ Rolling analysis complete!")
    return results.astype(float)

## 📊 Execute Rolling Analysis

Run the rolling window analysis to track model performance over time.

In [ ]:
# Execute rolling analysis with 60-day windows
window_size = 60
rolling_r2 = rolling_r2_scores(X_all, Y_all, window=window_size)

print(f"\n📈 Rolling R² Analysis Results:")
print(f"   • Window size: {window_size} trading days")
print(f"   • Analysis period: {rolling_r2.index.min()} to {rolling_r2.index.max()}")
print(f"   • Total observations: {len(rolling_r2)}")

# Summary statistics
print(f"\n📊 Rolling R² Summary Statistics:")
summary_stats = rolling_r2.describe()
print(summary_stats.round(3))

## 📈 Visualization & Export

Generate and save rolling R² charts for all EM indices.

In [ ]:
# Create output directory
output_dir = "../output/plots"
os.makedirs(output_dir, exist_ok=True)

print(f"📊 Generating rolling R² visualizations for {len(rolling_r2.columns)} EM indices...\n")

# Generate and save charts for each EM index
for col in rolling_r2.columns:
    plt.figure(figsize=(12, 6))
    
    # Plot rolling R²
    plt.plot(rolling_r2.index, rolling_r2[col], linewidth=2, alpha=0.8)
    
    # Add mean line
    mean_r2 = rolling_r2[col].mean()
    plt.axhline(y=mean_r2, color='red', linestyle='--', alpha=0.7, 
                label=f'Mean R² = {mean_r2:.3f}')
    
    # Formatting
    plt.title(f'Rolling R²: {col} vs Macro Factors ({window_size}-day PCA Model)', 
              fontsize=14, pad=20)
    plt.xlabel('Date')
    plt.ylabel('R² Score')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    
    # Save plot
    filename = f"rolling_r2_{col.replace('/', '_').replace(' ', '_')}.png"
    filepath = os.path.join(output_dir, filename)
    plt.savefig(filepath, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ {col}: Mean R² = {mean_r2:.3f}, Chart saved to {filename}")

# Create comprehensive summary plot
plt.figure(figsize=(14, 8))
for col in rolling_r2.columns:
    plt.plot(rolling_r2.index, rolling_r2[col], label=col, linewidth=1.5, alpha=0.8)

plt.title(f'Rolling R² Comparison: All EM Indices vs Macro Factors ({window_size}-day PCA Model)', 
          fontsize=14, pad=20)
plt.xlabel('Date')
plt.ylabel('R² Score')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save summary plot
summary_filename = "rolling_r2_all_indices_comparison.png"
summary_filepath = os.path.join(output_dir, summary_filename)
plt.savefig(summary_filepath, dpi=300, bbox_inches='tight')
plt.show()

print(f"\n💾 All visualizations saved to: {output_dir}")
print(f"📊 Summary chart: {summary_filename}")
print(f"🎯 Rolling analysis complete for {len(rolling_r2.columns)} EM indices!")